In [ ]:
import numpy as np
import pandas as pd
import networkx as nx # https://networkx.org/documentation/stable/index.html

In [ ]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

n

In [ ]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

I am not sure if it is really necessary to create the matrices A and w, if we use the networkx library. w might be usefull, but maybe A is not needed. I just tried it out before discovering the networkx package...
The code like this gives an (n-1)x(n-1) data frame where only the upper triangle is filled. If we want to represent it differently (e.g. symmetric matrix, boolean matrix, 2Dnumpy array, real upper triangular matrix etc) more transformation is needed.

I think for w it makes sense to use the matrix, because we only need it for calculating f. We have to be carefull with index, because the matrix has dimension (n-1)x(n-1) because the diagonal of the n x n matrix would just be 0 (connecting each node with itself costs nothing). Better to work with the row and column index then...

In [ ]:
# create adjacency matrix as data frame of upper triangular matrix
A = df.pivot(index='n1', columns='n2', values='e') #gives upper triangular data frame
A

In [ ]:
# create weight matrix as data frame of upper triangular matrix
w = df.pivot(index='n1', columns='n2', values='w') #gives upper triangular data frame
w

In [ ]:
w.loc[2,3] # weight of edge connecting node 2 and 3

In [ ]:
np.nansum(A.loc[4,])+np.nansum(A.loc[:,4]) # this should give us d(node4), as it is an upper triangular symmetric matrix. 
# But I don't know if we need this, as it is probably implemented in networkx.

In [ ]:
# try out how networkx works
# create empty graph
g = nx.Graph()

# get currently used edges (plus their weight, if we want)
current_e = df.loc[df["e"]==1][["n1", "n2", "w"]].to_numpy()
# fill graph by loading it with edges (with weights)
g.add_weighted_edges_from(current_e, weight = "w")

In [ ]:
# we can find out which nodes are currently connected
# should give us a list of nodes that are connected
d = list(nx.connected_components(g))
# d contains disconnected subgraphs
# d[0] contains the biggest subgraph
d

In [ ]:
# try this function with a completely disconnected graph to see the difference
g_disconnected = nx.Graph()
g_disconnected.add_nodes_from(range(1, n+1))

In [ ]:
def is_splex(G, s): # alternatively, can use the components as input, then have to get node degree from somewhere else 
    components = list(nx.connected_components(G))
    
    break_flag = False
    print(components)
    for c in components: # loop through components
        n_nodes = len(c)
        for n in list(c): # loop through nodes in components. It is a set and we have to transform it into a list
            print(G.degree[n])
            if(G.degree[n]<n_nodes-s):
                break_flag = True
                break
        if break_flag == True: # must be true for all nodes
            break
            
    return not(break_flag)

In [ ]:
is_splex(g, 170)

In [ ]:
# try looping through components. Will need more testing
for i in d1[0]:
    rem = df.loc[((df["n1"]==i) | (df["n2"]==i)) & (df["e"]==1)][["w"]].sum()
    print(rem)

In [ ]:
class Vertex:
    def __init__(self, name: int):
        self.name: int = name 
        self.degree: int = 0
        self.edge_weight: int = 0
        self.edges: set["Edge"] = set()

    def __str__(self) -> str:
        return str(self.name)

    def add_edge(self, edge: "Edge") -> None:
        self.edges.add(edge)
        self.degree += 1
        self.edge_weight += edge.get_weight()

    def get_degree(self) -> int:
        return self.degree
    
    def get_edge_weight(self) -> int:
        return self.edge_weight
    
    def get_edges(self) -> set["Edge"]:
        return self.edges
    
    def get_name(self) -> int:
        return self.name

class Edge:
    def __init__(self, v1: Vertex, v2: Vertex, weight: int):
        self.vertices = {v1, v2}
        self.weight: int = weight
    
    def __str__(self) -> str:
        return "[" + str(self.vertices[0]) + "-" + str(self.vertices[1]) + "]"
    
    def get_vertices(self) -> set[Vertex]:
        return self.vertices
    
    def get_weight(self) -> int:
        return self.weight

class Plex:
    def __init__(self, vertices):
        self.edges = set[Edge]()
        self.vertices = set[Vertex]()
        for vertex in vertices:
            self.add_vertex(vertex)

    def __str__(self) -> str:
        return "{" + (', '.join(list(map(str, self.vertices)))) + "}"

    def add_vertex(self, vertex: Vertex) -> None:
        self.vertices.add(vertex)
        
    def add_edge(self, edge: Edge) -> None:
        self.edges.add(edge)
        v1 = edge.get_vertices()[0]
        if self.vertices.contains(v1):
            v1.add_edge(edge)
        
        v2 = edge.get_vertices()[1]
        if self.vertices.contains(v2):
            v2.add_edge(edge)
        
        # TODO: More elegant solution?
    
    def get_vertices(self) -> set[Vertex]:
        return self.vertices
    
    def merge(self, other: "Plex") -> "Plex":
        merged = Plex(self.vertices.union(other.vertices))
        merged.edges = self.edges.union(other.edges)

        return merged
    
    def contains(self, vertexName) -> bool:
        for vertex in self.vertices:
            vertex.get_name == vertexName
    
class Graph:
    def __init__(self, vertices: set[Vertex]):
        self.plexs: set[Plex] = set()
        self.assignments: dict[int, Plex] = dict()
        self.total_weight = 0 # Not real weight, but if we allow negative weights it should not matter for the optimization problem
        for vertex in vertices:
            plex = Plex({vertex})
            self.plexs.add(plex)
            self.assignments[vertex.get_name()] = plex

    def __str__(self) -> str:
        return '\n'.join(list(map(str, self.plexs)))

    def merge(self, plex1: Plex, plex2: Plex):
        merged = plex1.merge(plex2)
        self.plexs.remove(plex1)
        self.plexs.remove(plex2)
        for vertex in merged.get_vertices():
            self.assignments[vertex] = merged

    def add_edge(self, edge: Edge) -> None:
        self.total_weight += edge.get_weight()
        
        participating_plexs = set(map(lambda v: self.assignments[v.get_name()], edge.get_vertices()))
        for p in participating_plexs:
            p.add_edge(edge)

    def get_plex(self, name: str) -> Plex:
        return self.assignments[name]
    

In [ ]:
vertices = set()
for i in range(1, n+1):
    vertices.add(Vertex(i))

graph = Graph(vertices)

print(str(graph))


In [ ]:
existing_edges = df.loc[df["e"]==1].sort_values("w", ascending=False)
print(existing_edges)

current_g = g_disconnected.copy()
s_Plexs = list(nx.connected_components(current_g))
plex_assignment = dict(map(lambda plex: (next(iter(plex)), plex), s_Plexs))

remaining_edges = len(existing_edges)

# TODO: Costs of the edges connected to a node
# TODO: Degree of a node
# TODO: Set of changed edges
# TODO: Total cost

for index, row in existing_edges.iterrows():
    n1_plex = plex_assignment[row["n1"]]
    n2_plex = plex_assignment[row["n2"]]
    nodes = n1_plex.union(n2_plex)

    other_edges_to_add = existing_edges.loc[(df["n1"].isin(nodes)) & (df["n2"].isin(nodes))]
    
    #if len(other_edges_to_add) > 2:
    #    print(len(other_edges_to_add))
    #    for edge in other_edges_to_add.iterrows():
    #        print(edge)
    #        
    #    print()

    number_of_nodes = len(nodes)
    edges_missing = False

    for node in nodes:
        if current_g.degree(node) < number_of_nodes - s:
            edges_missing = True
            # TODO: Add other edges

    if not edges_missing:
        current_g.add_edge(row["n1"], row["n2"])
        
        s_Plexs.remove(n1_plex)
        if n1_plex != n2_plex:
            s_Plexs.remove(n2_plex)
        s_Plexs.append(nodes)
        
        for node in nodes:
            plex_assignment[node] = nodes
            plex_assignment[node] = nodes

        remaining_edges -= 1

print(s_Plexs)